In [26]:
import requests
import json
import sys
import time

def query_ollama_stream(prompt):
    """Interact with the Ollama server using streaming HTTP responses and overwrite output."""
    url = "http://localhost:11434/api/chat"  # Correct endpoint
    payload = {
        "model": "llama3.1",  # Replace with your desired model name
        "messages": [  # Correct key
            {
                "role": "user",
                "content": prompt
            }
        ]
    }
    headers = {
        "Content-Type": "application/json"
    }

    try:
        # Set stream=True to handle streamed responses
        with requests.post(url, json=payload, headers=headers, stream=True) as response:
            response.raise_for_status()  # Raise an HTTPError for bad responses

            full_response = ""  # Initialize a variable to store the complete response
            for line in response.iter_lines():
                if line:  # Skip empty lines
                    try:
                        # Decode the JSON line
                        json_line = json.loads(line.decode("utf-8"))
                        # Extract the assistant's content
                        content = json_line.get("message", {}).get("content", "")
                        if content:
                            full_response += content  # Append new content
                            # Overwrite the line with the updated response
                            sys.stdout.write(f"\r{full_response}")
                            sys.stdout.flush()
                    except json.JSONDecodeError as e:
                        print(f"Error decoding JSON line: {e}")

            print()  # Move to a new line after the response is complete
            return full_response
    except requests.exceptions.RequestException as e:
        raise Exception(f"Error querying Ollama: {e}")

# Example Usage
if __name__ == "__main__":
    prompt = "Hi"
    try:
        response = query_ollama_stream(prompt)
        print("Final Response:", response)
    except Exception as e:
        print(f"Error: {e}")


How's it going? Is there something I can help you with or would you like to chat?
Final Response: How's it going? Is there something I can help you with or would you like to chat?


In [ ]:
import spacy

# Load the spaCy language model
nlp = spacy.load('en_core_web_sm')

def process_job_description():
    # Step 1: Read the contents of the offer.txt file
    with open('offer.txt', 'r') as file:
        job_description = file.read()
    
    doc = nlp(job_description)
    keywords = [token.text for token in doc if token.is_alpha and not token.is_stop]
    return set(keywords)
    



{'SOAP', 'training', 'Information', 'MuleSoft', 'problem', 'APIs', 'broker', 'looking', 'project', 'microservices', 'language', 'Excellent', 'company', 'Degree', 'Communication', 'good', 'Physics', 'similar', 'Microsoft', 'Implementation', 'skills', 'opportunity', 'colleagues', 'office', 'Developer', 'orientation', 'effective', 'message', 'principles', 'following', 'Curiosity', 'documentation', 'Genoa', 'Research', 'study', 'months', 'APEX', 'recent', 'management', 'tools', 'fields', 'cycle', 'innovative', 'proactivity', 'graduates', 'Axpo', 'teams', 'area', 'Computer', 'relevant', 'experience', 'methodologies', 'software', 'Technology', 'Good', 'implemented', 'custom', 'process', 'technical', 'Interest', 'Science', 'Strong', 'Salesforce', 'Mathematics', 'job', 'science', 'new', 'applications', 'Knowledge', 'profile', 'Java', 'architecture', 'Azure', 'business', 'computer', 'resource', 'Italian', 'development', 'languages', 'success', 'Requirements', 'join', 'ensure', 'Developing', 'Vi

In [5]:
import ollama
def get_professional_skills(keywords):
    # Convert keywords to a single string
    keywords_str = ', '.join(keywords)
    
    # Define the prompt for the Ollama model
    prompt = f"Extract professional skills from the following keywords: {keywords_str}"
    
    # Call the Ollama model
    response = ollama.ask(prompt)
    
    # Extract professional skills from the response
    professional_skills = response.get('professional_skills', [])
    
    return professional_skills

In [7]:
# Example usage
if __name__ == "__main__":
    keywords = process_job_description()
    professional_skills = get_professional_skills(keywords)
    print(professional_skills)

AttributeError: module 'ollama' has no attribute 'ask'